In [ ]:
!pwd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pwd

/content


In [4]:
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib as plt
import pickle
import statistics
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
df_jan = pq.read_table('/content/drive/My Drive/Colab Notebooks/yellow_tripdata_2023-01.parquet').to_pandas()

In [6]:
df_feb = pq.read_table('/content/drive/My Drive/Colab Notebooks/yellow_tripdata_2023-02.parquet').to_pandas()

In [7]:
df_jan.shape

(3066766, 19)

In [8]:
df_feb.shape

(2913955, 19)

In [9]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [10]:
df_jan.dtypes

,0
VendorID,int64
tpep_pickup_datetime,datetime64[us]
tpep_dropoff_datetime,datetime64[us]
passenger_count,float64
trip_distance,float64
RatecodeID,float64
store_and_fwd_flag,object
PULocationID,int64
DOLocationID,int64
payment_type,int64


In [11]:
df_jan['duration'] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).dt.total_seconds()/60

In [12]:
df_jan['PULocationID'] =df_jan['PULocationID'].astype(str)

In [13]:
df_jan['DOLocationID'] =df_jan['DOLocationID'].astype(str)

In [14]:
df_feb['duration'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds()/60

In [15]:
df_feb['PULocationID'] =df_feb['PULocationID'].astype(str)

In [16]:
df_feb['DOLocationID'] =df_feb['DOLocationID'].astype(str)

In [17]:
statistics.stdev(df_jan.duration)

42.59435124195457

In [18]:
df_jan_model = df_jan[(df_jan['duration']>=1) & (df_jan['duration']<=60)]

In [19]:
df_feb_model = df_feb[(df_feb['duration']>=1) & (df_feb['duration']<=60)]

In [20]:
df_jan_model.shape[0]/df_jan.shape[0]

0.9812202822125979

In [21]:
df_jan_model.shape

(3009173, 20)

In [22]:
df_jan_model.isnull().sum()/df_jan_model.shape[0]

,0
VendorID,0.000000
tpep_pickup_datetime,0.000000
tpep_dropoff_datetime,0.000000
passenger_count,0.023629
trip_distance,0.000000
RatecodeID,0.023629
store_and_fwd_flag,0.023629
PULocationID,0.000000
DOLocationID,0.000000
payment_type,0.000000


In [23]:
df_feb_model.isnull().sum()/df_feb_model.shape[0]

,0
VendorID,0.000000
tpep_pickup_datetime,0.000000
tpep_dropoff_datetime,0.000000
passenger_count,0.026313
trip_distance,0.000000
RatecodeID,0.026313
store_and_fwd_flag,0.026313
PULocationID,0.000000
DOLocationID,0.000000
payment_type,0.000000


In [24]:
df_jan_model = df_jan_model.dropna()

In [25]:
df_feb_model = df_feb_model.dropna()

In [26]:
df_feb_model.shape

(2780802, 20)

In [27]:
df_jan_model = df_jan_model.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [28]:
df_feb_model = df_feb_model.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [29]:
df_feb_model.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [30]:
x_train = df_jan_model.drop(columns=['duration'])

In [31]:
y_train = df_jan_model['duration'].values

In [32]:
df_dict = x_train.to_dict(orient = 'records')

In [33]:
vectorizer = DictVectorizer()

In [34]:
vectorizer.fit(df_dict)

DictVectorizer()

In [35]:
feature_matrix = vectorizer.transform(df_dict)

In [36]:
x_pred = df_feb_model.drop(columns=['duration'])

In [37]:
y_pred_feb = df_feb_model['duration'].values

In [38]:
df_dict_feb = x_pred.to_dict(orient = 'records')

In [39]:
feature_matrix_feb = vectorizer.transform(df_dict_feb)

In [40]:
type(feature_matrix)

scipy.sparse._csr.csr_matrix

In [41]:
feature_matrix.shape

(2938068, 530)

In [42]:
lr = LinearRegression()
lr.fit(feature_matrix, y_train)

LinearRegression()

In [43]:
y_pred = lr.predict(feature_matrix)

In [44]:
y_pred_feb_pred = lr.predict(feature_matrix_feb)

In [ ]:
sns.distplot(y_pred, label = 'Prediction')
sns.distplot(y_train, label = 'Actual')
plt.legend()

In [45]:
np.sqrt(mean_squared_error(y_train, y_pred))

np.float64(4.790989348777257)

In [46]:
np.sqrt(mean_squared_error(y_pred_feb_pred, y_pred_feb))

np.float64(5.090150794692873)